In [1]:
import pandas as pd
import json 
import numpy as np

                    Dataset Games               

Carga y guardado de datos en el csv "Games"

In [34]:
# Cargar el archivo JSON y lo guardo en el DataFrame dfgames
data = []
# Abrir el archivo JSON
with open('output_steam_games.json', "r") as data_file:
  # Leer el contenido del archivo y convertirlo a un objeto Python
    for line in data_file:
        df = json.loads(line)
        data.append(df)

dfgames = pd.DataFrame(data)

In [35]:
# Explorar un poco el DataFrame
dfgames.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
# Elimino filas con valores nulos en cualquier columna
dfgames = dfgames.dropna()

In [ ]:
# Obsevor que columna voy a usar y cual tengo que eliminar
dfgames.columns

Index(['publisher', 'genres', 'app_name', 'title', 'url', 'release_date',
       'tags', 'reviews_url', 'specs', 'price', 'early_access', 'id',
       'developer'],
      dtype='object')

In [37]:
# Uso la funcion drop para eliminar las columnas que no necesito
dfgames = dfgames.drop(['publisher', 'reviews_url', 'url', 'tags', 'specs', 'app_name','early_access'], axis=1)

In [38]:
dfgames.head(3)

,genres,title,release_date,price,id,developer
88310,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,2018-01-04,4.99,761140,Kotoshiro
88311,"[Free to Play, Indie, RPG, Strategy]",Ironbound,2018-01-04,Free To Play,643980,Secret Level SRL
88312,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,2017-07-24,Free to Play,670290,Poolians.com


Generamos dummmies en la columna genres para dividirlo y crear diferentes generos

In [39]:
# Rellenar los valores faltantes con una lista vacía
dfgames['genres'].fillna('[]', inplace=True)

# Convertir la lista de géneros a una cadena separada por comas
dfgames['genres'] = dfgames['genres'].apply(lambda x: ', '.join(x) )

# Crear variables ficticias para los géneros
genres_dummies = dfgames['genres'].str.get_dummies(', ')


# Concatenar las variables ficticias con el DataFrame original
dfgames = pd.concat([dfgames, genres_dummies], axis=1)

# Eliminar la columna genres 
dfgames.drop(['genres'], axis=1, inplace=True)


Cambio la columna 'release_date' a year. Para trabajar unicamente el año en lugar de un datetime

In [40]:
# Reemplazar los valores no válidos por NaN
dfgames["release_date"] = pd.to_datetime(dfgames["release_date"], errors='coerce')
# Filtrar el DataFrame para obtener los registros válidos
df_filtered = dfgames.dropna(subset=["release_date"])
# Convertir la columna "release_date" a tipo datetime
dfgames["release_date"] = pd.to_datetime(dfgames["release_date"])
# Rellenando los valores faltantes en las columna'release_date' con la moda
columns_fill_mode = ['release_date']
for col in columns_fill_mode:
    dfgames[col].fillna(dfgames[col].mode()[0], inplace=True)

In [41]:
# Modelado y eliminación de la columna 'release_date'
dfgames['year'] = dfgames['release_date'].dt.year.astype('Int64')
dfgames.drop('release_date', axis=1, inplace=True)

In [42]:
dfgames.head(5)

,title,price,id,developer,Action,Adventure,Animation &amp; Modeling,Audio Production,Casual,Design &amp; Illustration,...,RPG,Racing,Simulation,Software Training,Sports,Strategy,Utilities,Video Production,Web Publishing,year
88310,Lost Summoner Kitty,4.99,761140,Kotoshiro,1,0,0,0,1,0,...,0,0,1,0,0,1,0,0,0,2018
88311,Ironbound,Free To Play,643980,Secret Level SRL,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,2018
88312,Real Pool 3D - Poolians,Free to Play,670290,Poolians.com,0,0,0,0,1,0,...,0,0,1,0,1,0,0,0,0,2017
88313,弹炸人2222,0.99,767400,彼岸领域,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,2017
88315,Battle Royale Trainer,3.99,772540,Trickjump Games Ltd,1,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,2018


In [50]:
# Normalizo varios valores en la columna 'price' por 'Free to Play'
dfgames['price'] = dfgames['price'].replace(['Free', 'Free To Play', 'Free Mod', 'Free Demo', 'Play for Free!', 'Free HITMAN™ Holiday Pack'], 'Free to Play')

In [46]:
# Buscar registros duplicados
duplicados = dfgames[dfgames.duplicated()]
print("\nRegistros duplicados:")
print(duplicados)


Registros duplicados:
                                   title  price      id      developer  \
102883  Wolfenstein II: The New Colossus  59.99  612880  Machine Games   

        Action  Adventure  Animation &amp; Modeling  Audio Production  Casual  \
102883       1          0                         0                 0       0   

        Design &amp; Illustration  ...  RPG  Racing  Simulation  \
102883                          0  ...    0       0           0   

        Software Training  Sports  Strategy  Utilities  Video Production  \
102883                  0       0         0          0                 0   

        Web Publishing  year  
102883               0  2017  

[1 rows x 26 columns]


In [47]:
# Eliminar las filas duplicadas basadas en ciertas columnas
dfgames_cleaned = dfgames.drop_duplicates(subset=['title', 'price', 'id', 'developer'], keep='first')

# Imprimir el DataFrame limpio
print(dfgames_cleaned)

                           title         price      id  \
88310        Lost Summoner Kitty          4.99  761140   
88311                  Ironbound  Free To Play  643980   
88312    Real Pool 3D - Poolians  Free to Play  670290   
88313                    弹炸人2222          0.99  767400   
88315      Battle Royale Trainer          3.99  772540   
...                          ...           ...     ...   
120439              Kebab it Up!          1.99  745400   
120440            Colony On Mars          1.99  773640   
120441  LOGistICAL: South Africa          4.99  733530   
120442             Russian Roads          1.99  610660   
120443       EXIT 2 - Directions          4.99  658870   

                       developer  Action  Adventure  Animation &amp; Modeling  \
88310                  Kotoshiro       1          0                         0   
88311           Secret Level SRL       0          0                         0   
88312               Poolians.com       0          0         

In [52]:
# Guardamos en un csv
dfgames.to_csv('Games.csv', index=False)

                    Dataset Items                     

Carga, transformacion y guardado en csv de la tabla items

In [2]:
# Cargar el archivo JSON y lo guardo en el DataFrame Items
# Abrir el archivo JSON
with open("australian_users_items.json", 'r', encoding='utf-8') as i:
    data = i.readlines()
    
# Carga de los datos en el DataFrame df_items
datos = [eval(line.strip()) for line in data]
df_items = pd.DataFrame(datos)


In [3]:
# Explorar un poco el DataFrame
df_items.head(3)

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."


In [4]:
#Verificamos el contenido de items para ver que quiero sacar
print(df_items['items'][0])

[{'item_id': '10', 'item_name': 'Counter-Strike', 'playtime_forever': 6, 'playtime_2weeks': 0}, {'item_id': '20', 'item_name': 'Team Fortress Classic', 'playtime_forever': 0, 'playtime_2weeks': 0}, {'item_id': '30', 'item_name': 'Day of Defeat', 'playtime_forever': 7, 'playtime_2weeks': 0}, {'item_id': '40', 'item_name': 'Deathmatch Classic', 'playtime_forever': 0, 'playtime_2weeks': 0}, {'item_id': '50', 'item_name': 'Half-Life: Opposing Force', 'playtime_forever': 0, 'playtime_2weeks': 0}, {'item_id': '60', 'item_name': 'Ricochet', 'playtime_forever': 0, 'playtime_2weeks': 0}, {'item_id': '70', 'item_name': 'Half-Life', 'playtime_forever': 0, 'playtime_2weeks': 0}, {'item_id': '130', 'item_name': 'Half-Life: Blue Shift', 'playtime_forever': 0, 'playtime_2weeks': 0}, {'item_id': '300', 'item_name': 'Day of Defeat: Source', 'playtime_forever': 4733, 'playtime_2weeks': 0}, {'item_id': '240', 'item_name': 'Counter-Strike: Source', 'playtime_forever': 1853, 'playtime_2weeks': 0}, {'item_i

In [5]:
# aplico la funcion lambda para desanidar las keys que busco con sus values
df_items['playtime_forever'] = df_items['items'].apply(lambda x: x[0].get('playtime_forever') if len(x) > 0 else 0)
df_items['id'] = df_items['items'].apply(lambda x: x[0].get('item_id') if len(x) > 0 else 0)
df_items['playtime_forever'] = df_items['playtime_forever'].astype(int)


In [6]:
df_items.head(3)

,user_id,items_count,steam_id,user_url,items,playtime_forever,id
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik...",6,10
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik...",0,10
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest...",923,1200


In [7]:
df_items.drop(['user_url','items'], axis=1, inplace=True)

In [8]:
# Buscar registros duplicados
duplicados = df_items[df_items.duplicated()]
print("\nRegistros duplicados:")
print(duplicados)


Registros duplicados:
                 user_id  items_count           steam_id  playtime_forever  \
865           bokkkbokkk            0  76561198006988360                 0   
1732              Nikiad          109  76561198084006094                 5   
2343   76561198079743094           48  76561198079743094                30   
2344          ImSeriouss           50  76561198055318375              4312   
2394   76561198069124937            0  76561198069124937                 0   
...                  ...          ...                ...               ...   
48944  76561198035336388            5  76561198035336388               959   
67355  76561198071790027            0  76561198071790027                 0   
69344        darkus0haos          122  76561198073621277                 0   
73424  76561198080057659           39  76561198080057659                 0   
77045         farquadian            0  76561198086134170                 0   

          id  
865        0  
1732      

In [9]:
# Eliminar las filas duplicadas basadas en el 'user_id'
df_items_sin_duplicados = df_items.drop_duplicates(subset=['user_id'], keep='first')

# Imprimir el nuevo DataFrame sin duplicados
print("DataFrame sin filas duplicadas:")
print(df_items_sin_duplicados)

DataFrame sin filas duplicadas:
                       user_id  items_count           steam_id  \
0            76561197970982479          277  76561197970982479   
1                      js41637          888  76561198035864385   
2                    evcentric          137  76561198007712555   
3                   Riot-Punch          328  76561197963445855   
4                        doctr          541  76561198002099482   
...                        ...          ...                ...   
88305        76561198323066619           22  76561198323066619   
88306        76561198326700687          177  76561198326700687   
88307  XxLaughingJackClown77xX            0  76561198328759259   
88308        76561198329548331            7  76561198329548331   
88309        edward_tremethick            0  76561198331598578   

       playtime_forever      id  
0                     6      10  
1                     0      10  
2                   923    1200  
3                     0      10  
4    

In [10]:
df_items.head(3)

,user_id,items_count,steam_id,playtime_forever,id
0,76561197970982479,277,76561197970982479,6,10
1,js41637,888,76561198035864385,0,10
2,evcentric,137,76561198007712555,923,1200


In [11]:
# Guardamos en un csv
df_items.to_csv('Items.csv', index=False)

                    Dataset Reviews                     

Carga, transformacion y guardado de la tabla reviews en csv

In [2]:
# Cargar el archivo JSON y lo guardo en el DataFrame Reviews
# Abrir el archivo JSON
with open('australian_user_reviews.json', 'r', encoding= 'utf-8') as f:
    data = f.readlines()

# Carga de los datos en el DataFrame df_review
datos = [eval(line.strip()) for line in data]
df_review = pd.DataFrame(datos)

In [3]:
# Explorar un poco el DataFrame
df_review.head(3)

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."


Desanidado de la columna "reviews"

In [3]:
#creo un df con el archivo review y accedo ala column que me interesa desanidar.
df = df_review.explode('reviews') 
#con esta expresion  creo un nuevo df donde esta la info desanidada en columnas
df_review = pd.concat([df.drop(['reviews'], axis=1), df['reviews'].apply(pd.Series)], axis=1) 

Extraeremos el año y eliminaremos la columna Posted

In [4]:
# se cre la columna year
df_review['posted_year'] = df_review['posted'].str.extract(r'(\d{4})')
#Se elimina la columna posted
df_review.drop(['posted'], axis=1, inplace=True)

In [7]:
df_review.head(3)

,user_id,user_url,funny,last_edited,item_id,helpful,recommend,review,0,posted_year
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,NaN,2011
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,,22200,No ratings yet,True,It's unique and worth a playthrough.,NaN,2011
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,NaN,2011


In [5]:
df_review.drop(['posted_year', 'funny', 'last_edited', 'user_url', 'helpful', 0], axis=1, inplace=True)

Implementacion del Sentiment Score

In [6]:
#importamos las librerias
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon', quiet=True)

# Crear una instancia del analizador de sentimientos de Vader
sia = SentimentIntensityAnalyzer()

# Definir una función para asignar el valor de 'sentiment_analysis' según el análisis de sentimiento
def assign_sentiment_analysis(review):
    if pd.isna(review):
        return 1  # Valor neutral si no hay reseña
    else:
        compound_score = sia.polarity_scores(review)['compound']
        if compound_score >= 0.05:
            return 2  # Positivo
        elif compound_score <= -0.05:
            return 0  # Malo
        else:
            return 1  # Neutral
        
# Aplicar la función para crear la nueva columna 'sentiment_analysis'
df_review['sentiment_analysis'] = df_review['review'].apply(assign_sentiment_analysis)


In [7]:
# Convertir la columna 'review' a tipo de dato str
df_review['review'] = df_review['review'].astype(str)

In [8]:
df_review.head(3)

,user_id,funny,last_edited,item_id,recommend,review,posted_year,sentiment_analysis
0,76561197970982479,,,1250,True,Simple yet with great replayability. In my opi...,2011,2
0,76561197970982479,,,22200,True,It's unique and worth a playthrough.,2011,2
0,76561197970982479,,,43110,True,Great atmosphere. The gunplay can be a bit chu...,2011,2


In [9]:
# Buscar registros duplicados
duplicados = df_review[df_review.duplicated()]
print("\nRegistros duplicados:")
print(duplicados)


Registros duplicados:
                 user_id funny                    last_edited item_id  \
456           bokkkbokkk                                       346110   
1182          ImSeriouss                                       218620   
1182          ImSeriouss                                       105600   
1182          ImSeriouss                                          570   
1182          ImSeriouss                                       211820   
...                  ...   ...                            ...     ...   
18234  76561198092022514                                       422400   
18234  76561198092022514                                       218620   
18234  76561198092022514                                       261820   
18234  76561198092022514                                       224260   
18309     SuchGayMuchWow        Last edited December 1, 2015.     730   

      recommend                                             review  \
456        True               

In [10]:
# Eliminar las filas duplicadas basadas en el 'user_id'
df_items_sin_duplicados = df_review.drop_duplicates(subset=['user_id'], keep='first')

# Imprimir el nuevo DataFrame sin duplicados
print("DataFrame sin filas duplicadas:")
print(df_items_sin_duplicados)

DataFrame sin filas duplicadas:
                 user_id                             funny last_edited  \
0      76561197970982479                                                 
1                js41637                                                 
2              evcentric                                                 
3                  doctr                                                 
4              maplemage  3 people found this review funny               
...                  ...                               ...         ...   
25794  76561198306599751                                                 
25795           Ghoustik                                                 
25796  76561198310819422  1 person found this review funny               
25797  76561198312638244                                                 
25798        LydiaMorley  1 person found this review funny               

      item_id recommend                                             review  \
0

In [26]:
# Guardamos en un csv
df_review.to_csv('Reviews.csv', index=False) 

In [1]:
import pandas as pd
import numpy as np

                Creacion del dataset final                  

In [37]:
# A travez de id podemos unir tablas para ahorrar espacio
# Convino las tablas previamente desanidadas
df_merge = pd.merge(dfgames, df_items, on='id') 

In [38]:
# Unimos el dataset convinado con el dataset Items
df_final = pd.merge(df_merge, df_review, on= 'user_id')

In [39]:
# Reviso las columnas del dataframe final
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52707 entries, 0 to 52706
Data columns (total 34 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   title                      52707 non-null  object 
 1   price                      52707 non-null  object 
 2   id                         52707 non-null  int64  
 3   developer                  52707 non-null  object 
 4   Action                     52707 non-null  int64  
 5   Adventure                  52707 non-null  int64  
 6   Animation &amp; Modeling   52707 non-null  int64  
 7   Audio Production           52707 non-null  int64  
 8   Casual                     52707 non-null  int64  
 9   Design &amp; Illustration  52707 non-null  int64  
 10  Early Access               52707 non-null  int64  
 11  Education                  52707 non-null  int64  
 12  Free to Play               52707 non-null  int64  
 13  Indie                      52707 non-null  int

In [40]:
df_final.shape

(52707, 34)

In [41]:
df_final.head()

,title,price,id,developer,Action,Adventure,Animation &amp; Modeling,Audio Production,Casual,Design &amp; Illustration,...,Web Publishing,year,user_id,items_count,steam_id,playtime_forever,item_id,recommend,review,sentiment_analysis
0,Half-Life,9.99,70,Valve,1,0,0,0,0,0,...,0,1998,kube134,476,76561198031442694,4,251990.0,True,It's good to be a magical queen... if you surv...,2
1,Half-Life,9.99,70,Valve,1,0,0,0,0,0,...,0,1998,76561198030567998,75,76561198030567998,21,332800.0,True,10/10 would take kids here for birthday,1
2,Half-Life,9.99,70,Valve,1,0,0,0,0,0,...,0,1998,76561198030567998,75,76561198030567998,21,319630.0,True,"Well for starters, when I write reviews they a...",2
3,Half-Life,9.99,70,Valve,1,0,0,0,0,0,...,0,1998,76561198030567998,75,76561198030567998,21,6020.0,True,10/10 lets you cut people into sushi if the pr...,0
4,Half-Life,9.99,70,Valve,1,0,0,0,0,0,...,0,1998,76561198030567998,75,76561198030567998,21,55230.0,True,"10/10 has Sasha Grey, so fun for whole family.",2


In [43]:
# Guardamos el dataset final
df_final.to_csv('data_final.csv', index=False) 